In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
def read_file(path,extension,blank_firstrow=False):
    if extension == 'csv':
        if blank_firstrow:
            df = pd.read_csv(path, sep = ';', skiprows = 1, escapechar='\n')
        else:
            df = pd.read_csv(path, sep = ';', escapechar='\n')
    elif extension == 'txt':
        df = pd.read_csv(path, sep = '\t')
    elif extension == 'xls':
        df = pd.read_excel(path, skiprows = 1)
    return df

In [3]:
def clean_file(df,state_colname):
#     Set column "State" as index
    df = df.set_index(state_colname)
    # Remove rank columns
    col_names = list(df)
    del_list = []
    for item in col_names:
        if 'rank' in item.lower():
            del_list.append(item)

    df.drop(del_list, axis=1, inplace=True)
    
    #check for extraneous characters in name of state
    #replace white space with underscore in name of state
    states = df.index
    states_list= states.tolist()
    index_clean = []
    invalidChars = string.punctuation

    for word in states_list:
        word= word.strip()
#         print(word)
#         if any(char in invalidChars for char in word):
        for char in word:
            if(char in invalidChars):
#             print(char)
                word= word.replace(char, '')
                word= word.replace(' ', '_')
            else:
                word= word.replace(' ', '_')
        index_clean.append(word)
    
    df.index = index_clean
    return df

In [4]:
df_edu = read_file('../data/csv/education.csv','csv',True)
df_reg = read_file('../data/csv/region.txt','txt')
df_area = read_file('../data/csv/area.csv','csv')
df_exp = read_file('../data/csv/life_expectancy.csv','csv')
df_crime = read_file('../data/csv/crime.csv','csv')
df_income = read_file('../data/csv/income.xls','xls',True)

In [5]:
pd.set_option('display.max_rows', None)

In [6]:
clean_file(df_edu, 'State,federal district,or territory')

,% High school graduateor higher,% Bachelor's degreeor higher,% Advanced degree
Montana,93.0%,30.7%,10.1%
New_Hampshire,92.8%,36.0%,13.8%
Minnesota,92.8%,34.8%,11.8%
Wyoming,92.8%,26.7%,9.3%
Alaska,92.4%,29.0%,10.4%
North_Dakota,92.3%,28.9%,7.8%
Vermont,92.3%,36.8%,15.0%
Maine,92.1%,30.3%,10.9%
Iowa,91.8%,27.7%,9.0%
Utah,91.8%,32.5%,11.0%


In [7]:
clean_file(df_reg, 'Name')

,Abb,Region,Division
Alabama,AL,South,East South Central
Alaska,AK,West,Pacific
Arizona,AZ,West,Mountain
Arkansas,AR,South,west south central
California,CA,West,Pacific
Colorado,CO,West,Mountain
Connecticut,CT,Northeast,New England
Delaware,DE,South,south atlantic
Florida,FL,South,South Atlantic
Georgia,GA,South,South Atlantic


In [8]:
clean_file(df_area, 'State')

,TotalSqMi,TotalKmQ,LandSqMi,LandKmQ,LandPer,WaterSqMi,WaterKmQ,WaterPer
Alaska,665384.04,1723337,570640.95,1477953,85.76,94743.10,245384,14.24
Texas,268596.46,695662,261231.71,676587,97.26,7364.75,19075,2.74
California,163694.74,423967,155779.22,403466,95.16,7915.52,20501,4.84
Montana,147039.71,380831,145545.80,376962,98.98,1493.91,3869,1.02
New_Mexico,121590.30,314917,121298.15,314161,99.76,292.15,757,0.24
Arizona,113990.30,295234,113594.08,294207,99.65,396.22,1026,0.35
Nevada,110571.82,286380,109781.18,284332,99.28,790.65,2048,0.72
Colorado,104093.67,269601,103641.89,268431,99.57,451.78,1170,0.43
Oregon,98378.54,254799,95988.01,248608,97.57,2390.53,6191,2.43
Wyoming,97813.01,253335,97093.14,251470,99.26,719.87,1864,0.74


In [9]:
clean_file(df_exp, 'State')

,LifeExp2018,LifeExp2010,MaleLifeExp,FemLifeExp
Hawaii,82.3,81.4,79.3,85.3
California,81.6,80.6,79.4,83.8
Puerto_Rico,81.3,78.7,77.6,84.7
New_York,81.3,80.3,79.0,83.4
US_Virgin_Islands,81.2,79.2,76.3,85.6
Minnesota,81.0,80.8,79.0,83.0
Connecticut,80.9,80.7,78.7,83.0
Guam,80.7,78.2,77.6,83.8
Colorado,80.5,80.1,78.5,82.5
Massachusetts,80.5,80.5,78.2,82.6


In [10]:
df_c = clean_file(df_crime, 'State')
# df_c.drop_duplicates()
df_c.drop(df_c.tail(11).index,inplace = True)
df_c

,Population(total inhabitants) (2015) [2],Murders andNonnegligentManslaughter(total deaths) (2015) [1],Murders(total deaths) (2015) [3],Gun Murders(total deaths) (2015) [3],GunOwnership(%) (2013) [4],"Murder andNonnegligentManslaughterRate(per 100,000) (2015)","Murder Rate(per 100,000) (2015)","GunMurder Rate(per 100,000) (2015)"
Alabama,"4,853,875",348,—[a],—[a],48.9,7.20,— [a],— [a]
Alaska,"737,709",59,57,39,61.7,8.00,7.7,5.3
Arizona,"6,817,565",306,278,171,32.3,4.50,4.1,2.5
Arkansas,"2,977,853",181,164,110,57.9,6.10,5.5,3.7
California,"38,993,940","1,861","1,861","1,275",20.1,4.80,4.8,3.3
Colorado,"5,448,819",176,176,115,34.3,3.20,3.2,2.1
Connecticut,"3,584,730",117,107,73,16.6,3.30,3.0,2.0
Delaware,"944,076",63,63,52,5.2,6.70,6.7,5.5
District_of_Columbia,"670,377",162,162,121,25.9,24.20,24.2,18.0
Florida,"20,244,914","1,041",— [b],— [b],32.5,5.10,— [b],— [b]


In [11]:
clean_file(df_income, 'State')

,Income2017,Income2016,Income2015,Income2014,Income2013,Income2012,Income2011,Income2010,Income2009,Income2008,Income2007
District_of_Columbia,82336,75506,75628,71648,67572,65246,66583,63124,59290,57936,54317
Maryland,80776,78945,75847,73971,72483,71836,70004,68854,69272,70545,68080
New_Jersey,80088,76126,72222,72919,70165,69667,67458,67681,68342,70373,67035
Hawaii,77765,74511,73486,69592,68020,66259,61821,63030,64098,67214,63746
Massachusetts,77385,75297,70628,69160,66768,65339,62859,62072,64081,65401,62365
Connecticut,74168,73433,71346,70048,67098,67276,65753,64032,67034,68595,65967
New_Hampshire,73381,70936,70303,66532,64230,63280,62647,61042,60567,63731,62369
Alaska,73181,76440,73355,71583,72237,67712,67825,64576,66953,68460,64333
California,71805,67739,64500,61933,60190,58328,57287,57708,58931,61021,59948
Virginia,71535,68114,66262,64902,62666,61741,61882,60674,59330,61233,59562
